In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Movies data joining").getOrCreate()

In [2]:
from pyspark.sql.types import  *
from pyspark.sql.functions import from_json, regexp_replace

In [3]:
hadoop_base_directory = 'hdfs://192.168.56.101:9000/obligatorio'

# Movies

In [4]:
hadoop_movies_data = f'{hadoop_base_directory}/datasets/movies_metadata.csv'

In [5]:
movies = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").option("escape","\"").option("quote", "\"").load(hadoop_movies_data)

In [6]:
movies.count()

45572

In [7]:
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
w = Window().orderBy(lit('A'))
movies = movies.withColumn("row_num", row_number().over(w))

In [8]:
genres_schema = ArrayType(
    StructType([StructField("id", IntegerType()), 
                StructField("name", StringType())]))

prod_companies_schema = ArrayType(
    StructType([StructField("name", StringType()),
                StructField("id", IntegerType())]))

prod_countries_schema = ArrayType(
    StructType([StructField("iso_3166_1", StringType()),
                StructField("name", StringType())]))

spoken_languages_schema = ArrayType(
    StructType([StructField("iso_639_1", StringType()),
                StructField("name", StringType())]))

In [9]:
a_adult = "adult"
a_belongs_to = "belongs_to_collection"
a_budget = "budget"
a_genres = "genres"
a_id = "movie_id"
a_original_language = "original_language"
a_original_title = "original_title"
a_overview = "overview"
a_popularity = "popularity"
a_prod_companies = "production_companies"
a_production_countries = "production_countries"
a_release_date = "release_date"
a_revenue = "revenue"
a_spoken_languages = "spoken_languages"
a_title = "title"
a_vote_average = "vote_average"
a_vote_count = "vote_count"

relevant_fields = [a_adult, a_budget, a_genres, a_id, a_original_language, a_original_title,
                   a_overview, a_popularity, a_prod_companies, a_production_countries, a_release_date,
                  a_revenue, a_spoken_languages, a_title]

In [10]:
genres_schema = ArrayType(
    StructType([StructField("id", IntegerType()), 
                StructField("name", StringType())]))

prod_companies_schema = ArrayType(
    StructType([StructField("name", StringType()),
                StructField("id", IntegerType())]))

prod_countries_schema = ArrayType(
    StructType([StructField("iso_3166_1", StringType()),
                StructField("name", StringType())]))

spoken_languages_schema = ArrayType(
    StructType([StructField("iso_639_1", StringType()),
                StructField("name", StringType())]))

In [11]:
movies = movies.withColumn(a_adult, (movies.adult).cast("Boolean"))\
         .withColumn(a_id, (movies.id).cast("Integer"))\
         .withColumn(a_budget, (movies.budget).cast("Integer"))\
         .withColumn(a_genres, from_json(movies.genres, genres_schema))\
         .withColumn(a_prod_companies, from_json(movies.production_companies, prod_companies_schema))\
         .withColumn(a_production_countries, from_json(movies.production_countries, prod_countries_schema))\
         .withColumn(a_spoken_languages, from_json(movies.spoken_languages, spoken_languages_schema))\
         .withColumn(a_popularity, (movies.popularity).cast("Float"))\
         .withColumn(a_release_date, (movies.release_date).cast("Date"))\
         .withColumn(a_revenue, (movies.revenue).cast("Integer"))\
         .withColumn(a_vote_average, (movies.vote_average).cast("Float"))\
         .withColumn(a_vote_count, (movies.vote_count).cast("Integer"))

In [12]:
movies = movies.na.drop(subset=[a_adult, a_id, a_budget, a_genres, a_prod_companies, a_production_countries, 
                   a_spoken_languages, a_popularity, a_revenue, a_vote_average, a_vote_count])

In [13]:
movies = movies.dropDuplicates(subset=[a_id])

In [14]:
movies.count()

45326

In [ ]:
#from pyspark.sql.functions import row_number,lit
#from pyspark.sql.window import Window
#w = Window().orderBy(lit('A'))
#movies = movies.withColumn("row_num", row_number().over(w))

In [15]:
movies.select(["id","title", "production_companies"]).show(truncate = False)

+----+-----------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id  |title                                                                              |production_companies                                                                                                                                                                                                                |
+----+-----------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148 |The Secret Life of Words          

In [16]:
movies.show(10, truncate = False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-------------------------------------------------------+-------------------------------------------------------------+---+---------+-----------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
movies.filter(movies['vote_count'].isNull()).select(["movie_id","production_companies","release_date"]).show()

+--------+--------------------+------------+
|movie_id|production_companies|release_date|
+--------+--------------------+------------+
+--------+--------------------+------------+



In [18]:
movies.count()

45326

In [19]:
def get_element_in_pos(struct_list, pos):
    return list([ x[pos] for x in struct_list])

In [20]:
from pyspark.sql.functions import udf

extract_name_udf = udf(lambda z: get_element_in_pos(z, 1), ArrayType(StringType()))
extract_prod_company_name = udf(lambda z: get_element_in_pos(z, 0), ArrayType(StringType()))

In [21]:
movies = movies.withColumn("genres", extract_name_udf(movies.genres))\
        .withColumn("production_companies", extract_prod_company_name(movies.production_companies))\
        .withColumn("production_countries", extract_name_udf(movies.production_countries))\
        .withColumn("spoken_languages", extract_name_udf(movies.spoken_languages))\

In [22]:
movies.dtypes

[('adult', 'boolean'),
 ('belongs_to_collection', 'string'),
 ('budget', 'int'),
 ('genres', 'array<string>'),
 ('homepage', 'string'),
 ('id', 'string'),
 ('imdb_id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'float'),
 ('poster_path', 'string'),
 ('production_companies', 'array<string>'),
 ('production_countries', 'array<string>'),
 ('release_date', 'date'),
 ('revenue', 'int'),
 ('runtime', 'string'),
 ('spoken_languages', 'array<string>'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('video', 'string'),
 ('vote_average', 'float'),
 ('vote_count', 'int'),
 ('row_num', 'int'),
 ('movie_id', 'int')]

In [23]:
movies.select(["id","title", "spoken_languages"]).show(truncate = False)

+----+-----------------------------------------------------------------------------------+-----------------------------+
|id  |title                                                                              |spoken_languages             |
+----+-----------------------------------------------------------------------------------+-----------------------------+
|148 |The Secret Life of Words                                                           |[English, Français, Español] |
|471 |Bandyta                                                                            |[Polski, English]            |
|496 |Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan|[English, עִבְרִית, қазақ]   |
|833 |Umberto D.                                                                         |[Italiano]                   |
|1088|Whale Rider                                                                        |[English, ]                  |
|1580|Rope                      

In [24]:
movies.count()

45326

In [25]:
movies.dtypes

[('adult', 'boolean'),
 ('belongs_to_collection', 'string'),
 ('budget', 'int'),
 ('genres', 'array<string>'),
 ('homepage', 'string'),
 ('id', 'string'),
 ('imdb_id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'float'),
 ('poster_path', 'string'),
 ('production_companies', 'array<string>'),
 ('production_countries', 'array<string>'),
 ('release_date', 'date'),
 ('revenue', 'int'),
 ('runtime', 'string'),
 ('spoken_languages', 'array<string>'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('video', 'string'),
 ('vote_average', 'float'),
 ('vote_count', 'int'),
 ('row_num', 'int'),
 ('movie_id', 'int')]

In [26]:
movies = movies[relevant_fields]

# Ratings

In [27]:
hadoop_ratings_addr = f'{hadoop_base_directory}/ratings'
hadoop_links_addr = f'{hadoop_base_directory}/datasets/links.csv'

In [28]:
ratings_schema = StructType([
    StructField(a_id, IntegerType(), True),
    StructField("rating", FloatType(), True)])

In [29]:
ratings = spark.read.format("csv").option("header", "true").schema(ratings_schema).load(hadoop_ratings_addr, header=False)
links = spark.read.format("csv").option("header", "true").load(hadoop_links_addr, header=True)

In [30]:
ratings = ratings.na.drop(subset=["movie_id"])
ratings = ratings.dropDuplicates(subset=['movie_id'])

In [31]:
links = links.withColumn(a_id, (links.movieId).cast("Integer"))\
             .withColumn("tmdbId", (links.tmdbId).cast("Integer"))[a_id, "tmdbId"]

In [32]:
links.show(10)

+--------+------+
|movie_id|tmdbId|
+--------+------+
|       1|   862|
|       2|  8844|
|       3| 15602|
|       4| 31357|
|       5| 11862|
|       6|   949|
|       7| 11860|
|       8| 45325|
|       9|  9091|
|      10|   710|
+--------+------+
only showing top 10 rows



In [33]:
links.count()

45843

In [34]:
links = links.na.drop(subset=[a_id])
links = links.na.drop(subset=["tmdbId"])
links = links.dropDuplicates(subset=[a_id])
links = links.dropDuplicates(subset=["tmdbId"])

In [35]:
links.count()

45594

In [36]:
ratings_links = ratings.join(links, on=[a_id])

In [37]:
ratings_links.show()

+--------+---------+------+
|movie_id|   rating|tmdbId|
+--------+---------+------+
|     148|2.9099462| 22279|
|     463|2.8119159|  4916|
|     471|3.6548176| 11934|
|     496|3.2919621| 83718|
|     833|2.7146547|  9308|
|    1088|3.2398107|    88|
|    1238|3.9629796| 11235|
|    1342| 2.963798|  9529|
|    1580|3.5733178|   607|
|    1591| 2.641602| 10336|
|    1645|3.5165899|  1813|
|    1829|3.0827587| 30265|
|    1959|3.6369784|   606|
|    2122|2.6345134| 10823|
|    2142|3.0380545| 10380|
|    2366|3.4740872|   244|
|    2659|3.2386363| 30168|
|    2866|3.6019714| 24153|
|    3175|3.5865502|   926|
|    3749|3.2683823| 47439|
+--------+---------+------+
only showing top 20 rows



In [38]:
ratings = ratings_links.withColumn(a_id, (ratings_links.tmdbId).cast("Integer"))\
                 .withColumn("rating", (ratings_links.rating).cast("Float"))[a_id, "rating"]

In [39]:
ratings.count()

44872

In [40]:
movies.count()

45326

In [41]:
movies = movies.join(ratings, on=[a_id], how='left')

In [42]:
movies.count()

45326

# Keywords - Para busqueda por indexacion

In [43]:
hadoop_keywords_addr = f'{hadoop_base_directory}/datasets/keywords.csv'

In [44]:
keywords = spark.read.format("csv").option("header", "true").option("escape","\"").load(hadoop_keywords_addr)

In [45]:
a_keywords = "keywords"

keywords_schema = ArrayType(
    StructType([StructField("id", IntegerType()), 
                StructField("name", StringType())]))

In [46]:
extract_name_udf = udf(lambda z: get_element_in_pos(z, 1), ArrayType(StringType()))

In [47]:
keywords = keywords.withColumn(a_id, (keywords.id).cast("Integer"))\
                   .withColumn(a_keywords, from_json(keywords.keywords, keywords_schema))[a_id, a_keywords]

In [48]:
keywords = keywords.dropna()
keywords = keywords.dropDuplicates(subset=[a_id])

In [49]:
keywords.count()

45323

In [50]:
keywords = keywords.withColumn(a_keywords, extract_name_udf(keywords.keywords))

In [51]:
keywords.count()

45323

In [52]:
keywords.show(10, truncate = False)

+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|movie_id|keywords                                                                                                                                                                                                                                                                          |
+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148     |[nurse, yugoslavia, factory worker, dubrovnik, depression, loss of lover, war crimes, factory, dying and death, torture, oil platfor

In [53]:
movies = movies.join(keywords, on=[a_id], how='left')

In [54]:
movies.count()

45326

In [55]:
movies.select([a_id, a_title, a_keywords]).show(truncate=False)

+--------+-----------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|movie_id|title                                                                              |keywords                                                                                                                                                                                                                                                                          |
+--------+-----------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Obtener Cast (Actores) de Credits

In [56]:
hadoop_credits_addr = f'{hadoop_base_directory}/datasets/credits.csv'

In [57]:
credits = spark.read.format("csv").option("header", "true").option("escape","\"").load(hadoop_credits_addr)
credits = credits.withColumn('cast', regexp_replace('cast', ': None', ": ''"))

In [58]:
cast_schema = ArrayType(
    StructType([StructField("cast_id", IntegerType()), 
                StructField("character", StringType()),
                StructField("credit_id", StringType(), True),
                StructField("gender", IntegerType(), True),
                StructField("id", IntegerType()),
                StructField("name", StringType()),
                StructField("order", IntegerType(), True),
                StructField("profile_path", StringType(), True),
               ]))

In [59]:
credits = credits.withColumn(a_id, (credits.id).cast("Integer"))\
                  .withColumn("cast_actors", from_json(credits.cast, cast_schema))[a_id, "cast_actors"]

In [60]:
credits.filter(credits[a_id].isNull()).show()

+--------+-----------+
|movie_id|cast_actors|
+--------+-----------+
+--------+-----------+



In [61]:
credits.select(["cast_actors"]).show()

+--------------------+
|         cast_actors|
+--------------------+
|[[14, Woody (voic...|
|[[1, Alan Parrish...|
|[[2, Max Goldman,...|
|[[1, Savannah 'Va...|
|[[1, George Banks...|
|[[25, Lt. Vincent...|
|[[1, Linus Larrab...|
|[[2, Tom Sawyer, ...|
|[[1, Darren Franc...|
|[[1, James Bond, ...|
|[[1, Andrew Sheph...|
|[[9, Count Dracul...|
|[[1, Balto (voice...|
|[[1, Richard Nixo...|
|[[1, Morgan Adams...|
|[[4, Sam 'Ace' Ro...|
|[[6, Marianne Das...|
|[[42, Ted the Bel...|
|[[1, Ace Ventura,...|
|[[1, John, 52fe44...|
+--------------------+
only showing top 20 rows



In [62]:
credits = credits.na.drop(subset=['cast_actors'])
credits = credits.dropDuplicates(subset=[a_id])

In [63]:
extract_cast_name_udf = udf(lambda z: get_element_in_pos(z, 5), ArrayType(StringType()))

In [64]:
credits = credits.withColumn("cast_actors", extract_cast_name_udf(credits.cast_actors))

In [65]:
movies = movies.join(credits, on=[a_id], how='left')

In [66]:
movies.dtypes

[('movie_id', 'int'),
 ('adult', 'boolean'),
 ('budget', 'int'),
 ('genres', 'array<string>'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'float'),
 ('production_companies', 'array<string>'),
 ('production_countries', 'array<string>'),
 ('release_date', 'date'),
 ('revenue', 'int'),
 ('spoken_languages', 'array<string>'),
 ('title', 'string'),
 ('rating', 'float'),
 ('keywords', 'array<string>'),
 ('cast_actors', 'array<string>')]

In [67]:
movies.select(['cast_actors']).count()

45326

In [68]:
from pyspark.sql.functions import to_json, spark_partition_id, collect_list, col, struct

! rm -r movies_denormalized

movies.select(to_json(struct(*movies.columns)).alias("json"))\
    .groupBy(spark_partition_id())\
    .agg(collect_list("json").alias("json_list"))\
    .select(col("json_list").cast("string"))\
    .write.mode('overwrite').text('movies_denormalized')

In [69]:
! ./to_json.sh

In [70]:
hadoop_dest_folder_csv = 'hdfs://192.168.56.101:9000/obligatorio/'
movies_aggregated_dir = 'movies_aggregated'

In [ ]:
movies.write.mode('overwrite').json(f'{hadoop_dest_folder_csv}/{movies_aggregated_dir}')

In [ ]:
spark.stop()